In [41]:
import argparse
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LogisticRegression, LassoCV, ElasticNetCV
import argparse
from sklearn.utils import resample
import random
from collections import defaultdict, Counter
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score
import os

In [42]:
radiomics = pd.read_csv('/Users/laiacoronassala/Desktop/test cohorts olivia/MSKCC_R1B25.csv')
clinical = pd.read_csv('/Users/laiacoronassala/Desktop/test cohorts olivia/MSKCC_clinical_data.csv')

In [43]:
vars = [feat for feat in radiomics.keys() if 'original' in feat and 'diagnostics' not in feat] #ignoring "diagnostic information"
vars = ['Patient'] + vars #m'he de quedar amb aquesta columna, que m'indica ID de pacient
#crea un nou dataframe
rad_preprocessed = radiomics[vars].copy()
rad_preprocessed

,Patient,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,190420,0.877550,0.534736,27.599701,51.613674,56.356011,55.865911,63.031738,63.158531,43129.000000,...,37.969822,0.008819,6.162580,0.016254,9.295673e+05,14.153084,0.000186,72.908070,0.002352,0.075415
1,190420,0.600763,0.439556,17.329976,39.426096,44.283180,41.484937,30.413813,45.088801,10957.541667,...,56.098751,0.013452,5.574282,0.020464,1.960033e+05,3.888219,0.000719,64.930376,0.003051,0.324943
2,190516,0.612463,0.515625,16.752503,32.489683,34.481879,36.055513,23.537205,38.327536,7063.166667,...,48.520949,0.017108,5.340128,0.015222,1.530659e+05,2.548458,0.001412,40.340368,0.002928,0.382777
3,190549,0.820435,0.711017,17.083134,24.026350,29.068884,25.612497,25.079872,29.342802,5253.708333,...,188.074931,0.005333,6.153506,0.144508,3.123649e+03,0.374020,0.002758,417.297658,0.053324,1.562471
4,190622,0.825258,0.419338,8.824400,21.043659,21.260292,23.769729,21.095023,25.396850,1814.125000,...,56.570035,0.019785,5.559335,0.088828,3.407958e+03,0.326107,0.006947,115.203564,0.022760,1.939239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,348038,0.860673,0.648708,13.170264,20.302289,23.600847,23.769729,21.213203,25.079872,2841.083333,...,80.576945,0.010537,5.739733,0.079930,6.112311e+03,0.424134,0.004524,124.637461,0.019880,1.718892
325,190757,0.897771,0.558370,41.304763,73.973777,84.876381,85.000000,77.897368,88.971906,111470.250000,...,90.367621,0.004416,6.667123,0.012519,2.130179e+06,13.580529,0.000105,123.473015,0.002250,0.077140
326,347879,0.696466,0.538020,25.749724,47.860151,53.851648,51.195703,39.560081,54.083269,28324.875000,...,60.388268,0.008637,6.183731,0.011665,6.223233e+05,6.890468,0.000346,78.612235,0.003351,0.133698
327,348016,0.773213,0.661137,18.098114,27.374243,34.669872,26.907248,32.280025,37.456642,6670.833333,...,6.957770,0.070130,4.963071,0.008955,2.094889e+05,14.842422,0.001602,7.745738,0.014258,0.037094


In [44]:
#preparem les dades clíniques
#no ens interesa saber la lesió, de manera que ho eliminem del ID del pacient
clinical = clinical[['radiology_accession_number','clinical_benefit_rate']]
clinical = clinical.dropna(subset=['clinical_benefit_rate']) #si no tenim la info de la ground truth no volem aquest paceint
#Eliminen els pacients duplicats
clinical=clinical.drop_duplicates(subset=['radiology_accession_number']) 
GT_distribution = clinical['clinical_benefit_rate'].value_counts()
print("distribution of clinical benefit (ground truth) per RECIST:", GT_distribution)
clinical.rename(columns={'clinical_benefit_rate': 'GT'}, inplace=True)
clinical.rename(columns={'radiology_accession_number': 'Patient'}, inplace=True)
clinical

distribution of clinical benefit (ground truth) per RECIST: clinical_benefit_rate
0    116
1     72
Name: count, dtype: int64


,Patient,GT
0,190323,1
1,190342,1
2,190346,0
3,190351,0
4,190361,1
...,...,...
183,348063,1
184,348064,1
185,348066,0
186,348067,0


In [45]:
#Selecting three largest lesions for each patient and storing df
rad_3largest = rad_preprocessed.groupby('Patient', group_keys=False).apply(lambda x: x.sort_values('original_shape_MeshVolume', ascending=False).head(3)).reset_index(drop=True)

#Creating the column "tot_vol" which stores the variable of the total volume for the patient (total volume=sum of total volume of 3 largest lesions)
rad_3largest['tot_vol']=rad_3largest.groupby('Patient')['original_shape_MeshVolume'].transform('sum')

#Creating the column "weight" which is original_shape_MeshVolume/tot_vol for every lesion
rad_3largest['weight']=rad_3largest['original_shape_MeshVolume']/rad_3largest['tot_vol']
rad_3largest

,Patient,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,tot_vol,weight
0,190323,0.562091,0.498507,29.057507,58.289117,45.177428,56.302753,65.306967,65.429351,36484.666667,...,6.330821,0.031938,2.705611e+05,5.172009,0.000323,104.747194,0.006037,0.160577,36484.666667,1.000000
1,190342,0.678143,0.531421,7.408408,13.940762,16.155494,13.453624,11.704700,18.027756,459.791667,...,5.407865,0.172632,1.565790e+02,0.104275,0.026236,157.399952,0.059273,4.802729,459.791667,1.000000
2,190346,0.770957,0.688643,25.111323,36.464919,35.693137,43.600459,36.235342,44.810713,17190.458333,...,6.200036,0.087554,4.301871e+04,1.638274,0.000603,292.266579,0.019139,0.606705,25199.250000,0.682181
3,190346,0.909853,0.668802,19.593408,29.296287,36.138622,37.336309,33.837849,38.923001,8008.791667,...,5.740267,0.026460,6.283554e+04,1.826530,0.001622,60.617679,0.007501,0.432981,25199.250000,0.317819
4,190351,0.538801,0.407113,9.849586,24.193735,17.492856,22.803509,21.095023,26.758176,1734.791667,...,5.600487,0.101425,2.268862e+03,0.339834,0.006917,134.003039,0.035486,1.774899,1734.791667,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,348067,0.809452,0.251756,11.464243,45.537103,41.593269,46.872167,51.739733,51.778374,10665.125000,...,5.591025,0.012605,2.784003e+05,28.800746,0.000779,15.858952,0.006875,0.041340,83008.125000,0.128483
315,348067,0.665135,0.453696,16.935501,37.327852,40.249224,34.985711,45.000000,45.099889,8700.250000,...,5.398846,0.011095,2.720275e+05,36.234202,0.000963,8.682923,0.009924,0.023012,83008.125000,0.104812
316,348069,0.743596,0.446020,36.929895,82.798692,96.176920,104.694795,93.434469,108.967885,100648.291667,...,6.811511,0.011054,2.692317e+06,8.166046,0.000111,159.247426,0.001164,0.203760,106424.250000,0.945727
317,348069,0.711031,0.197108,8.577518,43.516742,33.941125,30.805844,44.384682,46.421978,3541.791667,...,5.195192,0.020084,3.905072e+04,6.279084,0.002929,11.658868,0.014698,0.081080,106424.250000,0.033280


In [46]:
#storing shape radiomic features in a temporary df because they don't have to be multiplied
shape_feat = [feat for feat in rad_3largest.keys() if 'original_shape' in feat] #ignoring "diagnostic information"
shape_feat = ['Patient'] + shape_feat + ['tot_vol']
shape_rad_3largest= rad_3largest[shape_feat].copy()
shape_rad_3largest

,Patient,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,tot_vol
0,190323,0.562091,0.498507,29.057507,58.289117,45.177428,56.302753,65.306967,65.429351,36484.666667,32.763811,0.589561,9023.230922,0.247316,36540.0,36484.666667
1,190342,0.678143,0.531421,7.408408,13.940762,16.155494,13.453624,11.704700,18.027756,459.791667,9.453833,0.675652,426.384939,0.927344,475.0,459.791667
2,190346,0.770957,0.688643,25.111323,36.464919,35.693137,43.600459,36.235342,44.810713,17190.458333,28.112884,0.692025,4654.654874,0.270770,17235.0,25199.250000
3,190346,0.909853,0.668802,19.593408,29.296287,36.138622,37.336309,33.837849,38.923001,8008.791667,26.655319,0.609081,3178.241250,0.396844,8050.0,25199.250000
4,190351,0.538801,0.407113,9.849586,24.193735,17.492856,22.803509,21.095023,26.758176,1734.791667,13.035619,0.666025,1048.314801,0.604289,1755.0,1734.791667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,348067,0.809452,0.251756,11.464243,45.537103,41.593269,46.872167,51.739733,51.778374,10665.125000,36.860119,0.572994,4089.262096,0.383424,10710.0,83008.125000
315,348067,0.665135,0.453696,16.935501,37.327852,40.249224,34.985711,45.000000,45.099889,8700.250000,24.828079,0.627323,3260.975664,0.374814,8743.0,83008.125000
316,348069,0.743596,0.446020,36.929895,82.798692,96.176920,104.694795,93.434469,108.967885,100648.291667,61.568757,0.488301,21428.966489,0.212909,100775.0,106424.250000
317,348069,0.711031,0.197108,8.577518,43.516742,33.941125,30.805844,44.384682,46.421978,3541.791667,30.941736,0.418551,2684.630787,0.757987,3585.0,106424.250000


In [47]:
#we compute non-shape features df, which is the one that will be weighted
non_shape_feat=[feat for feat in rad_3largest.keys() if not 'original_shape' in feat and not 'tot_vol' in feat]
non_shape_feat=rad_3largest[non_shape_feat].copy()
non_shape_feat

,Patient,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Mean,...,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,weight
0,190323,9.0,85.0,145984989.0,2.432507,33.0,12.937301,177.0,26.550559,47.591762,...,0.004692,6.330821,0.031938,2.705611e+05,5.172009,0.000323,104.747194,0.006037,0.160577,1.000000
1,190342,-156.6,49.6,3610082.0,3.511602,95.5,4.389522,99.0,61.694316,-36.450526,...,0.017399,5.407865,0.172632,1.565790e+02,0.104275,0.026236,157.399952,0.059273,4.802729,1.000000
2,190346,-88.0,52.0,96642967.0,2.772429,38.0,11.893035,115.0,47.830649,1.401393,...,0.005208,6.200036,0.087554,4.301871e+04,1.638274,0.000603,292.266579,0.019139,0.606705,0.682181
3,190346,-7.0,71.0,20408971.0,2.426692,40.0,9.838143,124.0,26.363729,34.739503,...,0.011850,5.740267,0.026460,6.283554e+04,1.826530,0.001622,60.617679,0.007501,0.432981,0.317819
4,190351,-110.6,44.6,9031358.0,3.137008,70.0,6.266154,89.0,51.680263,-16.395442,...,0.013296,5.600487,0.101425,2.268862e+03,0.339834,0.006917,134.003039,0.035486,1.774899,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,348067,28.0,76.0,31067506.0,1.702543,25.0,3.791111,177.0,14.878021,50.490196,...,0.056280,5.591025,0.012605,2.784003e+05,28.800746,0.000779,15.858952,0.006875,0.041340,0.128483
315,348067,21.0,65.0,16772560.0,1.604513,23.0,3.511299,133.0,13.935493,40.142972,...,0.035621,5.398846,0.011095,2.720275e+05,36.234202,0.000963,8.682923,0.009924,0.023012,0.104812
316,348069,35.0,89.0,447382139.0,1.991758,27.0,30.786379,195.0,17.971392,61.082977,...,0.002602,6.811511,0.011054,2.692317e+06,8.166046,0.000111,159.247426,0.001164,0.203760,0.945727
317,348069,67.0,128.0,37805842.0,2.041539,35.0,3.026981,164.0,19.791748,99.771827,...,0.062338,5.195192,0.020084,3.905072e+04,6.279084,0.002929,11.658868,0.014698,0.081080,0.033280


In [48]:
#Now we multiply every feature value by the corresponding weight
final_non_shape_feat=[feat for feat in non_shape_feat.keys() if not 'Patient' in feat and not 'weight' in feat]
final_non_shape_feat=non_shape_feat[final_non_shape_feat].copy()
weighted_feat=final_non_shape_feat.multiply(non_shape_feat['weight'], axis=0)

In [49]:
rad_df=pd.concat([shape_rad_3largest,weighted_feat],axis=1)
rad_df

,Patient,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,190323,0.562091,0.498507,29.057507,58.289117,45.177428,56.302753,65.306967,65.429351,36484.666667,...,93.341932,0.004692,6.330821,0.031938,2.705611e+05,5.172009,0.000323,104.747194,0.006037,0.160577
1,190342,0.678143,0.531421,7.408408,13.940762,16.155494,13.453624,11.704700,18.027756,459.791667,...,86.247490,0.017399,5.407865,0.172632,1.565790e+02,0.104275,0.026236,157.399952,0.059273,4.802729
2,190346,0.770957,0.688643,25.111323,36.464919,35.693137,43.600459,36.235342,44.810713,17190.458333,...,119.256988,0.003553,4.229549,0.059728,2.934656e+04,1.117600,0.000411,199.378809,0.013056,0.413883
3,190346,0.909853,0.668802,19.593408,29.296287,36.138622,37.336309,33.837849,38.923001,8008.791667,...,17.535982,0.003766,1.824364,0.008409,1.997031e+04,0.580505,0.000515,19.265429,0.002384,0.137609
4,190351,0.538801,0.407113,9.849586,24.193735,17.492856,22.803509,21.095023,26.758176,1734.791667,...,75.336556,0.013296,5.600487,0.101425,2.268862e+03,0.339834,0.006917,134.003039,0.035486,1.774899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,348067,0.809452,0.251756,11.464243,45.537103,41.593269,46.872167,51.739733,51.778374,10665.125000,...,0.915753,0.007231,0.718351,0.001620,3.576968e+04,3.700403,0.000100,2.037604,0.000883,0.005311
315,348067,0.665135,0.453696,16.935501,37.327852,40.249224,34.985711,45.000000,45.099889,8700.250000,...,0.386999,0.003734,0.565864,0.001163,2.851176e+04,3.797780,0.000101,0.910075,0.001040,0.002412
316,348069,0.743596,0.446020,36.929895,82.798692,96.176920,104.694795,93.434469,108.967885,100648.291667,...,154.208007,0.002461,6.441830,0.010454,2.546197e+06,7.722851,0.000105,150.604598,0.001101,0.192701
317,348069,0.711031,0.197108,8.577518,43.516742,33.941125,30.805844,44.384682,46.421978,3541.791667,...,0.235258,0.002075,0.172896,0.000668,1.299605e+03,0.208967,0.000097,0.388006,0.000489,0.002698


In [50]:
final_rad_df=rad_df.groupby('Patient').sum().reset_index()
final_rad_df

,Patient,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,190323,0.562091,0.498507,29.057507,58.289117,45.177428,56.302753,65.306967,65.429351,36484.666667,...,93.341932,0.004692,6.330821,0.031938,2.705611e+05,5.172009,0.000323,104.747194,0.006037,0.160577
1,190342,0.678143,0.531421,7.408408,13.940762,16.155494,13.453624,11.704700,18.027756,459.791667,...,86.247490,0.017399,5.407865,0.172632,1.565790e+02,0.104275,0.026236,157.399952,0.059273,4.802729
2,190346,1.680810,1.357445,44.704731,65.761206,71.831759,80.936768,70.073190,83.733714,25199.250000,...,136.792970,0.007319,6.053912,0.068137,4.931687e+04,1.698105,0.000927,218.644238,0.015440,0.551492
3,190351,0.538801,0.407113,9.849586,24.193735,17.492856,22.803509,21.095023,26.758176,1734.791667,...,75.336556,0.013296,5.600487,0.101425,2.268862e+03,0.339834,0.006917,134.003039,0.035486,1.774899
4,190361,2.196505,1.784966,64.721888,112.974992,135.918397,118.812292,101.357114,138.669142,45849.708333,...,84.643661,0.008491,6.202406,0.024627,1.818689e+05,5.040357,0.000562,114.719067,0.005807,0.284508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,348063,0.807094,0.489707,10.914534,22.287872,22.847319,20.124612,23.409400,24.718414,2555.666667,...,34.254986,0.029924,5.468794,0.078682,5.285497e+03,0.974032,0.004203,71.521417,0.028418,0.637352
182,348064,0.578672,0.432684,24.536803,56.708366,47.507894,51.623638,62.289646,67.904344,29512.166667,...,93.161165,0.006147,6.206384,0.024285,2.569039e+05,4.203178,0.000406,111.781034,0.004823,0.223333
183,348066,0.924592,0.570816,11.746630,20.578647,22.825424,24.413111,20.808652,25.495098,2672.500000,...,78.723794,0.010080,5.823576,0.079777,4.745570e+03,0.457092,0.004587,126.747332,0.018506,1.387929
184,348067,2.316555,1.390786,66.805747,138.904829,143.826361,144.483751,161.400386,163.579088,83008.125000,...,38.067129,0.015141,6.152920,0.008823,2.086377e+06,21.512343,0.000312,52.362513,0.003303,0.049401


In [51]:
final_rad_df.to_csv('3largest_MSKCC.csv')

In [52]:
all = final_rad_df.merge(clinical, how= 'inner',on = ['Patient'])
y_test = all['GT']
traincolumns = [col for col in all.columns if 'Patient' not in col and 'GT' not in col]
X = all[traincolumns]
scaler=StandardScaler()
X_test=scaler.fit_transform(X)

In [53]:
import pandas as pd
train = pd.read_csv('/Users/laiacoronassala/Desktop/VHIO/codisdef/aggregationmethods/186/3largest_biological.csv')
y_train = train['GT']
cols_train = [col for col in train.columns if 'Patient' not in col and 'GT' not in col and 'Unnamed' not in col]
X_train = train[cols_train]
X_train= scaler.fit_transform(X_train)

In [54]:
import warnings
from sklearn.exceptions import ConvergenceWarning
def final_model(X_train, y_train, X_test, y_test):
    # Usar un contexto para manejar advertencias
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)

        # Entrenar el modelo
        model = LogisticRegression(penalty='elasticnet', solver='saga', C=0.1, 
                                   l1_ratio=0.5, max_iter=500)
        model.fit(X_train, y_train)

        # Realizar predicciones y calcular AUC
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_pred_proba)

    return print(auc)

# Train the final model and export results

final_results = final_model(X_train, y_train, X_test, y_test)

0.4789823008849558
